# Chuẩn bị dữ liệu cho Naive Bayes

Xử lý dữ liệu từ `raw_data_bayes.csv` với format:
- `disease_id`: ID bệnh (D001, D002, ...)
- `disease_name`: Tên bệnh (Cảm lạnh, Cúm, ...)
- `symptoms`: Triệu chứng (cách nhau bởi dấu phẩy)

In [1]:
import pandas as pd
import json
import os

print("✅ Libraries loaded")

✅ Libraries loaded


## 1. Đọc dữ liệu từ CSV

In [2]:
print("📂 Đọc dữ liệu từ CSV...\n")

# Đọc file CSV
input_csv = "../../data/raw/raw_data_bayes.csv"

try:
    df_raw = pd.read_csv(input_csv)
    print(f"✓ Đã đọc {len(df_raw)} dòng từ '{input_csv}'")
    
    # Kiểm tra columns
    print(f"\n📋 Columns: {df_raw.columns.tolist()}")
    
    # Hiển thị mẫu
    print(f"\n📊 Dữ liệu mẫu (5 dòng đầu):")
    print(df_raw.head())
    
    # Kiểm tra missing values
    print(f"\n🔍 Missing values:")
    print(df_raw.isnull().sum())
    
except FileNotFoundError:
    print(f"❌ Không tìm thấy file '{input_csv}'")
    print("   Vui lòng kiểm tra đường dẫn file.")
except Exception as e:
    print(f"❌ Lỗi khi đọc file: {e}")

📂 Đọc dữ liệu từ CSV...

✓ Đã đọc 2173 dòng từ '../../data/raw/raw_data_bayes.csv'

📋 Columns: ['disease_id', 'disease_name', 'symptoms']

📊 Dữ liệu mẫu (5 dòng đầu):
  disease_id           disease_name  \
0       D001  Cảm lạnh thông thường   
1       D001  Cảm lạnh thông thường   
2       D001  Cảm lạnh thông thường   
3       D001  Cảm lạnh thông thường   
4       D001  Cảm lạnh thông thường   

                                       symptoms  
0           Chảy_nước_mũi, Sốt, Đau_nhức_cơ_thể  
1                Nghẹt_mũi, Khàn_giọng, Ho_khan  
2           Hắt_hơi_liên_tục, Đau_xoang, Sổ_mũi  
3          Sốt_nhẹ, Đau_đầu_âm_ỉ, Chảy_mũi_lỏng  
4  Mất_khứu_giác, Ho_có_đờm, Nghẹt_mũi_khó_chịu  

🔍 Missing values:
disease_id      0
disease_name    0
symptoms        0
dtype: int64


## 2. Xử lý và làm sạch dữ liệu

In [3]:
print("\n🔄 Xử lý dữ liệu...\n")

# Đổi tên cột cho dễ đọc (nếu cần)
# df_raw.columns = ['disease_id', 'disease', 'symptoms']  # Uncomment nếu cần

# Kiểm tra columns
required_cols = ['disease_id', 'disease_name', 'symptoms']
if not all(col in df_raw.columns for col in required_cols):
    print(f"❌ Lỗi: File CSV phải có các cột: {required_cols}")
    print(f"   Tìm thấy: {df_raw.columns.tolist()}")
else:
    # Xử lý dữ liệu
    disease_data = []
    
    for idx, row in df_raw.iterrows():
        disease_id = row['disease_id']
        disease_name = row['disease_name']
        symptoms_str = str(row['symptoms'])
        
        # Bỏ qua nếu thiếu dữ liệu
        if pd.isna(disease_name) or pd.isna(symptoms_str) or not symptoms_str:
            print(f"⚠️  Bỏ qua dòng {idx}: thiếu dữ liệu")
            continue
        
        # Tách triệu chứng (cách nhau bởi dấu phẩy)
        symptoms_list_raw = symptoms_str.split(',')
        
        # Làm sạch: loại bỏ khoảng trắng
        symptoms_list = [s.strip() for s in symptoms_list_raw if s.strip()]
        
        # Thêm vào danh sách
        if symptoms_list:
            disease_data.append({
                'disease_id': disease_id,
                'disease': disease_name,
                'symptoms': symptoms_list
            })
    
    print(f"✓ Đã xử lý {len(disease_data)} mẫu dữ liệu")
    
    # Thống kê
    print(f"\n📊 Thống kê:")
    unique_diseases = set([d['disease'] for d in disease_data])
    all_symptoms_set = set()
    for d in disease_data:
        all_symptoms_set.update(d['symptoms'])
    
    print(f"   - Số mẫu: {len(disease_data)}")
    print(f"   - Số bệnh: {len(unique_diseases)}")
    print(f"   - Số triệu chứng: {len(all_symptoms_set)}")
    
    # Hiển thị phân bố bệnh
    print(f"\n📋 Phân bố mẫu theo bệnh:")
    disease_counts = {}
    for d in disease_data:
        disease_counts[d['disease']] = disease_counts.get(d['disease'], 0) + 1
    
    for disease, count in sorted(disease_counts.items(), key=lambda x: (x[1], x[0])):
        print(f"   - {disease:<40} {count:>3} mẫu")


🔄 Xử lý dữ liệu...

✓ Đã xử lý 2173 mẫu dữ liệu

📊 Thống kê:
   - Số mẫu: 2173
   - Số bệnh: 21
   - Số triệu chứng: 625

📋 Phân bố mẫu theo bệnh:
   - Bệnh Phổi tắc_nghẽn_mạn_tính (COPD)        1 mẫu
   - Bệnh Gút (Gout)                           92 mẫu
   - Ung thư (Gan, Phổi, Dạ dày)               96 mẫu
   - Bệnh Lao phổi                            100 mẫu
   - Bệnh Tay - Chân - Miệng                  100 mẫu
   - Bệnh Tiêu chảy cấp                       100 mẫu
   - Cảm cúm                                  100 mẫu
   - Cảm lạnh thông thường                    100 mẫu
   - Sốt rét                                  100 mẫu
   - Sốt xuất huyết                           100 mẫu
   - Viêm Kết Mạc                             100 mẫu
   - Viêm họng và Viêm Amidan cấp             100 mẫu
   - Viêm phế quản cấp                        100 mẫu
   - Viêm loét dạ dày và Tá tràng             101 mẫu
   - Đái tháo đường (Tiểu đường)              110 mẫu
   - Viêm khớp mạn tính                   

## 3. Tạo danh sách tất cả triệu chứng

In [4]:
print("\n🔧 Tạo danh sách triệu chứng...\n")

# Tạo tập hợp tất cả triệu chứng
all_symptoms = set()
for item in disease_data:
    all_symptoms.update(item['symptoms'])

# Sắp xếp
all_symptoms = sorted(list(all_symptoms))

print(f"✓ Tổng số triệu chứng: {len(all_symptoms)}")

# Hiển thị 20 triệu chứng đầu
print(f"\n📋 Danh sách triệu chứng (20 đầu tiên):")
for i, symptom in enumerate(all_symptoms[:20], 1):
    print(f"   {i:3d}. {symptom}")

if len(all_symptoms) > 20:
    print(f"   ... và {len(all_symptoms) - 20} triệu chứng khác")


🔧 Tạo danh sách triệu chứng...

✓ Tổng số triệu chứng: 625

📋 Danh sách triệu chứng (20 đầu tiên):
     1. Ban_đỏ
     2. Bị_bít_mũi
     3. Bị_nhức_đầu
     4. Bị_sốt
     5. Bị_sổ_mũi
     6. Chảy_mũi
     7. Chảy_mũi_lỏng
     8. Chảy_mũi_trong
     9. Chảy_nước_mũi
    10. Chảy_nước_mũi_liên_tục
    11. Chảy_nước_mũi_nhiều
    12. Chảy_nước_mũi_ít
    13. Có_sốt
    14. Cảm_giác_sốt
    15. Giảm_khứu_giác
    16. Giảm_ăn
    17. Giọng_khàn
    18. Ho
    19. Ho_có_đờm
    20. Ho_dữ_dội
   ... và 605 triệu chứng khác


## 4. Chuyển sang binary format

In [5]:
print("\n🔄 Chuyển sang binary format...\n")

# Tạo binary matrix
binary_data = []

for item in disease_data:
    # Tạo row: 1 nếu có triệu chứng, 0 nếu không
    row = [1 if symptom in item['symptoms'] else 0 for symptom in all_symptoms]
    
    # Thêm disease_id và disease_name
    row_dict = {
        'disease_id': item['disease_id'],
        'disease': item['disease']
    }
    
    # Thêm từng triệu chứng
    for symptom, value in zip(all_symptoms, row):
        row_dict[symptom] = value
    
    binary_data.append(row_dict)

# Tạo DataFrame
df_binary = pd.DataFrame(binary_data)

print(f"✓ Đã tạo binary matrix: {df_binary.shape}")
print(f"   - Rows (samples): {df_binary.shape[0]}")
print(f"   - Columns (disease_id + disease + symptoms): {df_binary.shape[1]}")

# Hiển thị mẫu
print(f"\n📊 Binary data mẫu (5 dòng, 10 cột đầu):")
print(df_binary.iloc[:5, :10])


🔄 Chuyển sang binary format...

✓ Đã tạo binary matrix: (2173, 627)
   - Rows (samples): 2173
   - Columns (disease_id + disease + symptoms): 627

📊 Binary data mẫu (5 dòng, 10 cột đầu):
  disease_id                disease  Ban_đỏ  Bị_bít_mũi  Bị_nhức_đầu  Bị_sốt  \
0       D001  Cảm lạnh thông thường       0           0            0       0   
1       D001  Cảm lạnh thông thường       0           0            0       0   
2       D001  Cảm lạnh thông thường       0           0            0       0   
3       D001  Cảm lạnh thông thường       0           0            0       0   
4       D001  Cảm lạnh thông thường       0           0            0       0   

   Bị_sổ_mũi  Chảy_mũi  Chảy_mũi_lỏng  Chảy_mũi_trong  
0          0         0              0               0  
1          0         0              0               0  
2          0         0              0               0  
3          0         0              1               0  
4          0         0              0              

## 5. Lưu dữ liệu

In [6]:
print("\n💾 Lưu dữ liệu...\n")

# Tạo thư mục
os.makedirs("../../data/processed", exist_ok=True)

# 1. Lưu binary data (CSV)
output_csv = "../../data/processed/naive_bayes_data.csv"
df_binary.to_csv(output_csv, index=False, encoding="utf-8-sig")
print(f"✓ Đã lưu: {output_csv}")

# 2. Lưu symptom mapping (JSON)
output_mapping = "../../data/processed/symptom_mapping.json"
symptom_mapping = {
    "all_symptoms": all_symptoms,
    "diseases": sorted(list(set([d['disease'] for d in disease_data]))),
    "disease_ids": sorted(list(set([d['disease_id'] for d in disease_data]))),
    "n_samples": len(disease_data),
    "n_symptoms": len(all_symptoms),
    "n_diseases": len(set([d['disease'] for d in disease_data]))
}

with open(output_mapping, "w", encoding="utf-8") as f:
    json.dump(symptom_mapping, f, ensure_ascii=False, indent=2)

print(f"✓ Đã lưu: {output_mapping}")

# 3. Lưu disease mapping (JSON) - Map disease_id -> disease_name
output_disease_map = "../../data/processed/disease_mapping.json"
disease_mapping = {}
for item in disease_data:
    disease_mapping[item['disease_id']] = item['disease']

with open(output_disease_map, "w", encoding="utf-8") as f:
    json.dump(disease_mapping, f, ensure_ascii=False, indent=2)

print(f"✓ Đã lưu: {output_disease_map}")

print("\n" + "="*60)
print("✅ HOÀN TẤT XỬ LÝ DỮ LIỆU")
print("="*60)

print(f"\n📦 Files đã tạo:")
print(f"   1. {output_csv}")
print(f"   2. {output_mapping}")
print(f"   3. {output_disease_map}")

print(f"\n📊 Tóm tắt:")
print(f"   - Số mẫu: {symptom_mapping['n_samples']}")
print(f"   - Số bệnh: {symptom_mapping['n_diseases']}")
print(f"   - Số triệu chứng: {symptom_mapping['n_symptoms']}")

print(f"\n📝 Bước tiếp theo:")
print(f"   → Chạy notebook: 04_train_model_naive_bayes_optimized.ipynb")


💾 Lưu dữ liệu...

✓ Đã lưu: ../../data/processed/naive_bayes_data.csv
✓ Đã lưu: ../../data/processed/symptom_mapping.json
✓ Đã lưu: ../../data/processed/disease_mapping.json

✅ HOÀN TẤT XỬ LÝ DỮ LIỆU

📦 Files đã tạo:
   1. ../../data/processed/naive_bayes_data.csv
   2. ../../data/processed/symptom_mapping.json
   3. ../../data/processed/disease_mapping.json

📊 Tóm tắt:
   - Số mẫu: 2173
   - Số bệnh: 21
   - Số triệu chứng: 625

📝 Bước tiếp theo:
   → Chạy notebook: 04_train_model_naive_bayes_optimized.ipynb


## 6. Kiểm tra dữ liệu đã lưu

In [7]:
print("\n🔍 Kiểm tra dữ liệu đã lưu...\n")

# Load lại CSV
df_check = pd.read_csv(output_csv)
print(f"✓ CSV: {df_check.shape}")
print(f"  Columns: {df_check.columns[:5].tolist()} ... (+{len(df_check.columns)-5} more)")

# Load lại JSON
with open(output_mapping, "r", encoding="utf-8") as f:
    mapping_check = json.load(f)

print(f"\n✓ Symptom mapping:")
print(f"  Symptoms: {len(mapping_check['all_symptoms'])}")
print(f"  Diseases: {len(mapping_check['diseases'])}")

# Load disease mapping
with open(output_disease_map, "r", encoding="utf-8") as f:
    disease_map_check = json.load(f)

print(f"\n✓ Disease mapping: {len(disease_map_check)} entries")
print(f"\n  Ví dụ:")
for disease_id, disease_name in list(disease_map_check.items())[:5]:
    print(f"    {disease_id} -> {disease_name}")

print("\n✅ Tất cả files đều OK!")


🔍 Kiểm tra dữ liệu đã lưu...

✓ CSV: (2173, 627)
  Columns: ['disease_id', 'disease', 'Ban_đỏ', 'Bị_bít_mũi', 'Bị_nhức_đầu'] ... (+622 more)

✓ Symptom mapping:
  Symptoms: 625
  Diseases: 21

✓ Disease mapping: 20 entries

  Ví dụ:
    D001 -> Cảm lạnh thông thường
    D002 -> Cảm cúm
    D003 -> Viêm họng và Viêm Amidan cấp
    D004 -> Viêm phế quản cấp
    D005 -> Bệnh Lao phổi

✅ Tất cả files đều OK!
